## 간단한 Feature Engineering

---

학습기록을 적재중인 GCP spanner에서 윈도우 function 등을 지원하고 있지 않기 때문에 pandas 라이브러리를 사용. 데이터의 크기가 커서 GCP high memory instance 에서 진행.

| feature name | Description |
| --- | --- |
| lag_time | 현재 row와 사용자의  직전 학습기록 간의 시간 간격(초) |
| question_lag_time | 현재 row의 문제를 사용자가 얼마만에 다시 푸는지 시간 간격(초) |
| accuracy_per_question | 전체 유저 대상 문제의 평균 정답율(0~100 사이 정수 값으로 변경) |
| tag | 한 문제는 여러 알고리즘 태그를 갖는다. 문제들이 갖는 모든 태그 조합을 1~ 의 정수로 나타내고 부여된 태그가 없는 경우 0을 부여  |
| tag_lag_time | 유저가 현재 row 이전에 해당 태그를 얼마만에 다시 푸는지 시간 간격(초) |
| accuracy_per_tag | 전체 유저 대상 해당 태그의 평균 정답율(0~100 사이 정수 값으로 변경) |
| cumulated_points | result: 문제를 맞췄을때 1 틀렸을때 0<br>relative result: (100*result) - (accuracy_per_question)<br>cumulated_points: relative result를 누적한 값으로 시점에 유저의 상대적인 실력을 나타냄(정수 값으로 변경) |
| prior_correct_count | 유저가 해당 로우 이전까지 몇번의 맞은 학습기록이 있는지 |
| prior_count | 유저가 해당 로우 이전까지 몇번의 학습기록이 있는지 |
| prior_accuracy | 유저의 해당 로우 이전까지의 정답율 |
| prior_tag_correct_count | 유저가 해당 로우 이전까지 해당 태그를 가진 문제에 대해 몇번의 맞은 학습기록이 있는지 |
| prior_tag_count | 유저가 해당 로우 이전까지 해당 태그를 가진 문제에 대해 몇번의 학습기록이 있는지 |
| prior_tag_accuracy | 유저의 해당 로우 이전까지 해당 태그를 가진 문제에 대한 정답율 |

In [1]:
import os
import pandas as pd
import numpy as np
import json
from collections import defaultdict



In [2]:
df=pd.read_csv("/home/gcpwoong/data/lgbm_data_full/lgbm_data_n1.csv")
df=df.sort_values(["user_id","submittedTimestamp"])


,historyId,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries
0,47433752,000000,2557,1,1660030281,1,242853,2.5228
1,47433892,000000,7287,1,1660030387,1,45277,1.9571
2,47434954,000000,10172,1,1660031212,1,123178,2.4217
3,47435195,000000,10699,1,1660031392,1,17083,1.7208
4,47435494,000000,10171,1,1660031619,1,124606,3.0053
...,...,...,...,...,...,...,...,...
23999519,50219999,zzzzzzzzu2,15663,1,1665124657,9,9436,2.0177
23999520,50311067,zzzzzzzzu2,1182,1,1665316270,9,18158,2.2535
23999521,50353396,zzzzzzzzu2,1759,0,1665402290,11,17976,2.2445
23999522,50353974,zzzzzzzzu2,1759,1,1665402978,11,17976,2.2445


> json들 다시 만들기

In [3]:
# df=df.drop('historyId',axis=1)

# uniq_user = np.unique(df['user_id'])

# id2name={}
# for i,name in enumerate(uniq_user):
#     id2name[int(i)]=name
# name2id={}
# for k,v in id2name.items():
#     name2id[str(v)]=k
# df["user_id"]=list(map(lambda x:name2id[x],df["user_id"]))
# df

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries
0,15835,2579,0,1632714600,8,34903,2.9617
1,15835,2579,1,1632714674,8,34903,2.9617
2,15835,17071,0,1633091862,16,1337,4.2917
3,15835,17071,0,1633091880,16,1337,4.2917
4,15835,17071,1,1633279084,16,1337,4.2917
...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2.0177
23999520,79526,1182,1,1665316270,9,18158,2.2535
23999521,79526,1759,0,1665402290,11,17976,2.2445
23999522,79526,1759,1,1665402978,11,17976,2.2445


In [4]:
# with open('/home/gcpwoong/data/lgbm_data_full/id2name.json','w') as f:
#   json.dump(id2name, f, ensure_ascii=False, indent=4)
# with open('/home/gcpwoong/data/lgbm_data_full/name2id.json','w') as f:
#   json.dump(name2id, f, ensure_ascii=False, indent=4)

> 불러오기

In [3]:
import json
with open('/home/gcpwoong/data/lgbm_data_full/name2id.json', 'r') as f:
    name2id = json.load(f)

In [13]:
df

,historyId,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries
0,33774644,dlwoabsdk,2579,0,1632714600,8,34903,2.9617
1,33774671,dlwoabsdk,2579,1,1632714674,8,34903,2.9617
2,33953420,dlwoabsdk,17071,0,1633091862,16,1337,4.2917
3,33953432,dlwoabsdk,17071,0,1633091880,16,1337,4.2917
4,34029484,dlwoabsdk,17071,1,1633279084,16,1337,4.2917
...,...,...,...,...,...,...,...,...
23999519,50219999,zzzzzzzzu2,15663,1,1665124657,9,9436,2.0177
23999520,50311067,zzzzzzzzu2,1182,1,1665316270,9,18158,2.2535
23999521,50353396,zzzzzzzzu2,1759,0,1665402290,11,17976,2.2445
23999522,50353974,zzzzzzzzu2,1759,1,1665402978,11,17976,2.2445


In [4]:
df=df.drop('historyId',axis=1)
df["user_id"]=list(map(lambda x:name2id[x],df["user_id"]))
df

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries
0,0,2557,1,1660030281,1,242853,2.5228
1,0,7287,1,1660030387,1,45277,1.9571
2,0,10172,1,1660031212,1,123178,2.4217
3,0,10699,1,1660031392,1,17083,1.7208
4,0,10171,1,1660031619,1,124606,3.0053
...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2.0177
23999520,79526,1182,1,1665316270,9,18158,2.2535
23999521,79526,1759,0,1665402290,11,17976,2.2445
23999522,79526,1759,1,1665402978,11,17976,2.2445


In [5]:
df=df.astype({"averageTries":"int"})
df=df.sort_values(["user_id","submittedTimestamp"])
df["lag_time"]=df.groupby(["user_id"])["submittedTimestamp"].diff().fillna(0).astype(int)
df

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time
0,0,2557,1,1660030281,1,242853,2,0
1,0,7287,1,1660030387,1,45277,1,106
2,0,10172,1,1660031212,1,123178,2,825
3,0,10699,1,1660031392,1,17083,1,180
4,0,10171,1,1660031619,1,124606,3,227
...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2,64612
23999520,79526,1182,1,1665316270,9,18158,2,191613
23999521,79526,1759,0,1665402290,11,17976,2,86020
23999522,79526,1759,1,1665402978,11,17976,2,688


In [6]:
df["question_lag_time"]=df.groupby(["user_id","question_id"])["submittedTimestamp"].diff().fillna(0).astype(int)
df

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time
0,0,2557,1,1660030281,1,242853,2,0,0
1,0,7287,1,1660030387,1,45277,1,106,0
2,0,10172,1,1660031212,1,123178,2,825,0
3,0,10699,1,1660031392,1,17083,1,180,0
4,0,10171,1,1660031619,1,124606,3,227,0
...,...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2,64612,0
23999520,79526,1182,1,1665316270,9,18158,2,191613,0
23999521,79526,1759,0,1665402290,11,17976,2,86020,0
23999522,79526,1759,1,1665402978,11,17976,2,688,688


In [7]:
df.to_csv("/home/gcpwoong/data/lgbm_data_full/new_lgbm_data.csv",index=False)


In [8]:
accuracy_per_question=df.groupby("question_id")["result"].mean()
df['accuracy_per_question'] = df['question_id'].map(accuracy_per_question)
df['accuracy_per_question']=list(map(lambda x:round(100*x),df.accuracy_per_question.tolist()))
df

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question
0,0,2557,1,1660030281,1,242853,2,0,0,72
1,0,7287,1,1660030387,1,45277,1,106,0,60
2,0,10172,1,1660031212,1,123178,2,825,0,59
3,0,10699,1,1660031392,1,17083,1,180,0,64
4,0,10171,1,1660031619,1,124606,3,227,0,61
...,...,...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2,64612,0,71
23999520,79526,1182,1,1665316270,9,18158,2,191613,0,55
23999521,79526,1759,0,1665402290,11,17976,2,86020,0,58
23999522,79526,1759,1,1665402978,11,17976,2,688,688,58


In [9]:
with open('/home/gcpwoong/data/lgbm_data_full/tag_dic.json', 'r') as f:
    tag_dic = json.load(f)
df["tag"]=df["question_id"].astype(str).map(tag_dic)
df["tag"]=df["tag"].fillna(0).astype(int)
df

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,tag
0,0,2557,1,1660030281,1,242853,2,0,0,72,923
1,0,7287,1,1660030387,1,45277,1,106,0,60,923
2,0,10172,1,1660031212,1,123178,2,825,0,59,923
3,0,10699,1,1660031392,1,17083,1,180,0,64,923
4,0,10171,1,1660031619,1,124606,3,227,0,61,923
...,...,...,...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2,64612,0,71,2202
23999520,79526,1182,1,1665316270,9,18158,2,191613,0,55,3229
23999521,79526,1759,0,1665402290,11,17976,2,86020,0,58,1782
23999522,79526,1759,1,1665402978,11,17976,2,688,688,58,1782


In [10]:
df["tag_lag_time"]=df.groupby(["user_id","tag"])["submittedTimestamp"].diff().fillna(0).astype(int)
df

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,tag,tag_lag_time
0,0,2557,1,1660030281,1,242853,2,0,0,72,923,0
1,0,7287,1,1660030387,1,45277,1,106,0,60,923,106
2,0,10172,1,1660031212,1,123178,2,825,0,59,923,825
3,0,10699,1,1660031392,1,17083,1,180,0,64,923,180
4,0,10171,1,1660031619,1,124606,3,227,0,61,923,227
...,...,...,...,...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2,64612,0,71,2202,219725
23999520,79526,1182,1,1665316270,9,18158,2,191613,0,55,3229,256225
23999521,79526,1759,0,1665402290,11,17976,2,86020,0,58,1782,0
23999522,79526,1759,1,1665402978,11,17976,2,688,688,58,1782,688


In [11]:
df.to_csv("/home/gcpwoong/data/lgbm_data_full/new_lgbm_data.csv",index=False)

In [3]:
df=pd.read_csv("/home/gcpwoong/data/lgbm_data_full/new_lgbm_data.csv")


In [4]:
df

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,tag,tag_lag_time
0,0,2557,1,1660030281,1,242853,2,0,0,72,923,0
1,0,7287,1,1660030387,1,45277,1,106,0,60,923,106
2,0,10172,1,1660031212,1,123178,2,825,0,59,923,825
3,0,10699,1,1660031392,1,17083,1,180,0,64,923,180
4,0,10171,1,1660031619,1,124606,3,227,0,61,923,227
...,...,...,...,...,...,...,...,...,...,...,...,...
23987454,79526,15663,1,1665124657,9,9436,2,64612,0,71,2202,219725
23987455,79526,1182,1,1665316270,9,18158,2,191613,0,55,3229,256225
23987456,79526,1759,0,1665402290,11,17976,2,86020,0,58,1782,0
23987457,79526,1759,1,1665402978,11,17976,2,688,688,58,1782,688


In [12]:
tag_acc=df.groupby("tag")["result"].mean()
df['accuracy_per_tag'] = df['tag'].map(tag_acc)
df['accuracy_per_tag']=list(map(lambda x:round(100*x),df.accuracy_per_tag.tolist()))
df


,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,tag,tag_lag_time,accuracy_per_tag
0,0,2557,1,1660030281,1,242853,2,0,0,72,923,0,73
1,0,7287,1,1660030387,1,45277,1,106,0,60,923,106,73
2,0,10172,1,1660031212,1,123178,2,825,0,59,923,825,73
3,0,10699,1,1660031392,1,17083,1,180,0,64,923,180,73
4,0,10171,1,1660031619,1,124606,3,227,0,61,923,227,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2,64612,0,71,2202,219725,78
23999520,79526,1182,1,1665316270,9,18158,2,191613,0,55,3229,256225,67
23999521,79526,1759,0,1665402290,11,17976,2,86020,0,58,1782,0,58
23999522,79526,1759,1,1665402978,11,17976,2,688,688,58,1782,688,58


In [13]:
df["temp_for_relative"]=(100*df["result"]-df["accuracy_per_question"])
df["cumulated_points"]=df.groupby(["user_id"])["temp_for_relative"].cumsum().shift(fill_value=0)
df


,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,tag,tag_lag_time,accuracy_per_tag,temp_for_relative,cumulated_points
0,0,2557,1,1660030281,1,242853,2,0,0,72,923,0,73,28,0
1,0,7287,1,1660030387,1,45277,1,106,0,60,923,106,73,40,28
2,0,10172,1,1660031212,1,123178,2,825,0,59,923,825,73,41,68
3,0,10699,1,1660031392,1,17083,1,180,0,64,923,180,73,36,109
4,0,10171,1,1660031619,1,124606,3,227,0,61,923,227,73,39,145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2,64612,0,71,2202,219725,78,29,1456
23999520,79526,1182,1,1665316270,9,18158,2,191613,0,55,3229,256225,67,45,1485
23999521,79526,1759,0,1665402290,11,17976,2,86020,0,58,1782,0,58,-58,1530
23999522,79526,1759,1,1665402978,11,17976,2,688,688,58,1782,688,58,42,1472


In [14]:
# 이전 문제 정답 횟수
df['prior_correct_count'] = df.groupby('user_id')['result'].cumsum().shift(fill_value=0)
df['prior_count'] = df.groupby('user_id')['result'].cumcount().fillna(0)
df

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,tag,tag_lag_time,accuracy_per_tag,temp_for_relative,cumulated_points,prior_correct_count,prior_count
0,0,2557,1,1660030281,1,242853,2,0,0,72,923,0,73,28,0,0,0
1,0,7287,1,1660030387,1,45277,1,106,0,60,923,106,73,40,28,1,1
2,0,10172,1,1660031212,1,123178,2,825,0,59,923,825,73,41,68,2,2
3,0,10699,1,1660031392,1,17083,1,180,0,64,923,180,73,36,109,3,3
4,0,10171,1,1660031619,1,124606,3,227,0,61,923,227,73,39,145,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2,64612,0,71,2202,219725,78,29,1456,60,63
23999520,79526,1182,1,1665316270,9,18158,2,191613,0,55,3229,256225,67,45,1485,61,64
23999521,79526,1759,0,1665402290,11,17976,2,86020,0,58,1782,0,58,-58,1530,62,65
23999522,79526,1759,1,1665402978,11,17976,2,688,688,58,1782,688,58,42,1472,62,66


In [15]:
df['prior_accuracy'] = (df['prior_correct_count'] / df['prior_count']).fillna(0)
df

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,tag,tag_lag_time,accuracy_per_tag,temp_for_relative,cumulated_points,prior_correct_count,prior_count,prior_accuracy
0,0,2557,1,1660030281,1,242853,2,0,0,72,923,0,73,28,0,0,0,0.000000
1,0,7287,1,1660030387,1,45277,1,106,0,60,923,106,73,40,28,1,1,1.000000
2,0,10172,1,1660031212,1,123178,2,825,0,59,923,825,73,41,68,2,2,1.000000
3,0,10699,1,1660031392,1,17083,1,180,0,64,923,180,73,36,109,3,3,1.000000
4,0,10171,1,1660031619,1,124606,3,227,0,61,923,227,73,39,145,4,4,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2,64612,0,71,2202,219725,78,29,1456,60,63,0.952381
23999520,79526,1182,1,1665316270,9,18158,2,191613,0,55,3229,256225,67,45,1485,61,64,0.953125
23999521,79526,1759,0,1665402290,11,17976,2,86020,0,58,1782,0,58,-58,1530,62,65,0.953846
23999522,79526,1759,1,1665402978,11,17976,2,688,688,58,1782,688,58,42,1472,62,66,0.939394


In [16]:
# df['prior_correct_count'] = 
df["prior_tag_correct_count"]=df.groupby(['user_id',"tag"])['result'].cumsum().shift(fill_value=0)
df["prior_tag_count"]=df.groupby(['user_id',"tag"])['result'].cumcount().fillna(0)
df['prior_tag_accuracy'] = round(100*(df['prior_tag_correct_count'] / df['prior_tag_count']).fillna(0))
df


,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,...,tag_lag_time,accuracy_per_tag,temp_for_relative,cumulated_points,prior_correct_count,prior_count,prior_accuracy,prior_tag_correct_count,prior_tag_count,prior_tag_accuracy
0,0,2557,1,1660030281,1,242853,2,0,0,72,...,0,73,28,0,0,0,0.000000,0,0,0.0
1,0,7287,1,1660030387,1,45277,1,106,0,60,...,106,73,40,28,1,1,1.000000,1,1,100.0
2,0,10172,1,1660031212,1,123178,2,825,0,59,...,825,73,41,68,2,2,1.000000,2,2,100.0
3,0,10699,1,1660031392,1,17083,1,180,0,64,...,180,73,36,109,3,3,1.000000,3,3,100.0
4,0,10171,1,1660031619,1,124606,3,227,0,61,...,227,73,39,145,4,4,1.000000,4,4,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2,64612,0,71,...,219725,78,29,1456,60,63,0.952381,1,6,17.0
23999520,79526,1182,1,1665316270,9,18158,2,191613,0,55,...,256225,67,45,1485,61,64,0.953125,7,1,700.0
23999521,79526,1759,0,1665402290,11,17976,2,86020,0,58,...,0,58,-58,1530,62,65,0.953846,2,0,inf
23999522,79526,1759,1,1665402978,11,17976,2,688,688,58,...,688,58,42,1472,62,66,0.939394,0,1,0.0


In [22]:
df.loc[(df["user_id"]==14521)&(df["question_id"]==10872)]

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,...,tag_lag_time,accuracy_per_tag,temp_for_relative,cumulated_points,prior_correct_count,prior_count,prior_accuracy,prior_tag_correct_count,prior_tag_count,prior_tag_accuracy
3680929,14521,10872,0,1625055457,1,59138,1,0,0,77,...,0,77,-77,2786,86,0,inf,1,0,inf
13290472,14521,10872,1,1625055570,1,59138,1,113,113,77,...,113,77,23,-77,0,1,0.000000,0,1,0.0
21660258,14521,10872,1,1653982632,1,59138,1,3605226,28927062,77,...,28927062,77,23,1219,82,118,0.694915,1,2,50.0
20459855,14521,10872,1,1653982660,1,59138,1,28,28,77,...,28,77,23,1242,83,119,0.697479,2,3,67.0


In [17]:
idxs=df.loc[df["prior_tag_count"]<df["prior_tag_correct_count"]].index
df.loc[idxs, ['prior_tag_correct_count', 'prior_tag_accuracy']] = 0
idxs=df.loc[df["prior_count"]<df["prior_correct_count"]].index
df.loc[idxs, ['prior_correct_count', 'prior_accuracy']] = 0
df


,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,...,tag_lag_time,accuracy_per_tag,temp_for_relative,cumulated_points,prior_correct_count,prior_count,prior_accuracy,prior_tag_correct_count,prior_tag_count,prior_tag_accuracy
0,0,2557,1,1660030281,1,242853,2,0,0,72,...,0,73,28,0,0,0,0.000000,0,0,0.0
1,0,7287,1,1660030387,1,45277,1,106,0,60,...,106,73,40,28,1,1,1.000000,1,1,100.0
2,0,10172,1,1660031212,1,123178,2,825,0,59,...,825,73,41,68,2,2,1.000000,2,2,100.0
3,0,10699,1,1660031392,1,17083,1,180,0,64,...,180,73,36,109,3,3,1.000000,3,3,100.0
4,0,10171,1,1660031619,1,124606,3,227,0,61,...,227,73,39,145,4,4,1.000000,4,4,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,9436,2,64612,0,71,...,219725,78,29,1456,60,63,0.952381,1,6,17.0
23999520,79526,1182,1,1665316270,9,18158,2,191613,0,55,...,256225,67,45,1485,61,64,0.953125,0,1,0.0
23999521,79526,1759,0,1665402290,11,17976,2,86020,0,58,...,0,58,-58,1530,62,65,0.953846,0,0,0.0
23999522,79526,1759,1,1665402978,11,17976,2,688,688,58,...,688,58,42,1472,62,66,0.939394,0,1,0.0


In [24]:
df.loc[(df["user_id"]==14521)&(df["question_id"]==10872)]

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,...,tag_lag_time,accuracy_per_tag,temp_for_relative,cumulated_points,prior_correct_count,prior_count,prior_accuracy,prior_tag_correct_count,prior_tag_count,prior_tag_accuracy
3680929,14521,10872,0,1625055457,1,59138,1,0,0,77,...,0,77,-77,2786,0,0,0.000000,0,0,0.0
13290472,14521,10872,1,1625055570,1,59138,1,113,113,77,...,113,77,23,-77,0,1,0.000000,0,1,0.0
21660258,14521,10872,1,1653982632,1,59138,1,3605226,28927062,77,...,28927062,77,23,1219,82,118,0.694915,1,2,50.0
20459855,14521,10872,1,1653982660,1,59138,1,28,28,77,...,28,77,23,1242,83,119,0.697479,2,3,67.0


In [18]:
df['prior_accuracy'] = df['prior_accuracy'].replace(np.nan, 0)
df['prior_accuracy'].isna().sum()
df['prior_tag_accuracy'] = df['prior_tag_accuracy'].replace(np.nan, 0)
df['prior_tag_accuracy'].isna().sum()

0

In [19]:
df['prior_accuracy']=round(100*df['prior_accuracy'])

In [20]:
df.loc[(df["user_id"]==14521)&(df["question_id"]==10872)]


,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,...,tag_lag_time,accuracy_per_tag,temp_for_relative,cumulated_points,prior_correct_count,prior_count,prior_accuracy,prior_tag_correct_count,prior_tag_count,prior_tag_accuracy
4494384,14521,10872,0,1625055457,1,59138,1,0,0,77,...,0,77,-77,2786,0,0,0.0,0,0,0.0
4494385,14521,10872,1,1625055570,1,59138,1,113,113,77,...,113,77,23,-77,0,1,0.0,0,1,0.0
4494502,14521,10872,1,1653982632,1,59138,1,3605226,28927062,77,...,28927062,77,23,1219,82,118,69.0,1,2,50.0
4494503,14521,10872,1,1653982660,1,59138,1,28,28,77,...,28,77,23,1242,83,119,70.0,2,3,67.0


In [21]:
df['prior_tag_accuracy'] = df['prior_tag_accuracy'].astype(int)
df['prior_accuracy'] = df['prior_accuracy'].astype(int)


In [33]:
df.loc[(df["user_id"]==14521)&(df["question_id"]==10872)]

,user_id,question_id,result,submittedTimestamp,level_id,acceptedUserCount,averageTries,lag_time,question_lag_time,accuracy_per_question,...,tag_lag_time,accuracy_per_tag,temp_for_relative,cumulated_points,prior_correct_count,prior_count,prior_accuracy,prior_tag_correct_count,prior_tag_count,prior_tag_accuracy
3680929,14521,10872,0,1625055457,1,59138,1,0,0,77,...,0,77,-77,0,0,0,0,0,0,0
13290472,14521,10872,1,1625055570,1,59138,1,113,113,77,...,113,77,23,-77,0,1,0,0,1,0
21660258,14521,10872,1,1653982632,1,59138,1,3605226,28927062,77,...,28927062,77,23,1219,82,118,69,1,2,50
20459855,14521,10872,1,1653982660,1,59138,1,28,28,77,...,28,77,23,1242,83,119,70,2,3,67


In [24]:
df.to_csv("/home/gcpwoong/data/lgbm_data_full/new_lgbm_data.csv",index=False)

KeyboardInterrupt: 

In [23]:
user_start_id = df['user_id'].diff() > 0
features = ['cumulated_points',
            'prior_correct_count',
            'prior_count',
            'prior_accuracy',
            'prior_tag_correct_count',
            'prior_tag_count',
            'prior_tag_accuracy']
 
# 각 학생의 첫 row는 prior과 lagtime feature의 값을 0으로 초기화한다
df.loc[user_start_id, features] = 0

In [25]:
df.columns

Index(['user_id', 'question_id', 'result', 'submittedTimestamp', 'level_id',
       'acceptedUserCount', 'averageTries', 'lag_time', 'question_lag_time',
       'accuracy_per_question', 'tag', 'tag_lag_time', 'accuracy_per_tag',
       'temp_for_relative', 'cumulated_points', 'prior_correct_count',
       'prior_count', 'prior_accuracy', 'prior_tag_correct_count',
       'prior_tag_count', 'prior_tag_accuracy'],
      dtype='object')

In [26]:
df=df.drop(columns=["acceptedUserCount","temp_for_relative"])

In [27]:
df.to_csv("/home/gcpwoong/data/lgbm_data_full/new_lgbm_data.csv",index=False)

In [28]:
df

,user_id,question_id,result,submittedTimestamp,level_id,averageTries,lag_time,question_lag_time,accuracy_per_question,tag,tag_lag_time,accuracy_per_tag,cumulated_points,prior_correct_count,prior_count,prior_accuracy,prior_tag_correct_count,prior_tag_count,prior_tag_accuracy
0,0,2557,1,1660030281,1,2,0,0,72,923,0,73,0,0,0,0,0,0,0
1,0,7287,1,1660030387,1,1,106,0,60,923,106,73,28,1,1,100,1,1,100
2,0,10172,1,1660031212,1,2,825,0,59,923,825,73,68,2,2,100,2,2,100
3,0,10699,1,1660031392,1,1,180,0,64,923,180,73,109,3,3,100,3,3,100
4,0,10171,1,1660031619,1,3,227,0,61,923,227,73,145,4,4,100,4,4,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23999519,79526,15663,1,1665124657,9,2,64612,0,71,2202,219725,78,1456,60,63,95,1,6,17
23999520,79526,1182,1,1665316270,9,2,191613,0,55,3229,256225,67,1485,61,64,95,0,1,0
23999521,79526,1759,0,1665402290,11,2,86020,0,58,1782,0,58,1530,62,65,95,0,0,0
23999522,79526,1759,1,1665402978,11,2,688,688,58,1782,688,58,1472,62,66,94,0,1,0


In [ ]:
FEATS = ["question_id","level_id",
         "averageTries","lag_time","accuracy_per_question","tag","accuracy_per_tag",
         "cumulated_points","prior_correct_count","prior_count","prior_accuracy","prior_tag_correct_count",
         "prior_tag_count","prior_tag_accuracy"]